In [72]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
# from tqdm import tqdm
import time
import random
import lxml
import pandas as pd

In [73]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [74]:
# 출력 행 제한을 없애는 코드
pd.set_option('display.max_rows', 200)

### 날짜 생성 코드

In [75]:
import datetime

start_date = datetime.date(1995, 1, 3)
end_date = datetime.date(2024, 7, 31)

# 월-금요일에 해당하는 날짜 리스트 생성
weekdays = [0, 1, 2, 3, 4]  # 월요일=0, 화요일=1, ..., 금요일=4
date_list = []

current_date = start_date
while current_date <= end_date:
    if current_date.weekday() in weekdays:
        date_list.append(current_date)
    current_date += datetime.timedelta(days=1)

print(date_list)


[datetime.date(1995, 1, 3), datetime.date(1995, 1, 4), datetime.date(1995, 1, 5), datetime.date(1995, 1, 6), datetime.date(1995, 1, 9), datetime.date(1995, 1, 10), datetime.date(1995, 1, 11), datetime.date(1995, 1, 12), datetime.date(1995, 1, 13), datetime.date(1995, 1, 16), datetime.date(1995, 1, 17), datetime.date(1995, 1, 18), datetime.date(1995, 1, 19), datetime.date(1995, 1, 20), datetime.date(1995, 1, 23), datetime.date(1995, 1, 24), datetime.date(1995, 1, 25), datetime.date(1995, 1, 26), datetime.date(1995, 1, 27), datetime.date(1995, 1, 30), datetime.date(1995, 1, 31), datetime.date(1995, 2, 1), datetime.date(1995, 2, 2), datetime.date(1995, 2, 3), datetime.date(1995, 2, 6), datetime.date(1995, 2, 7), datetime.date(1995, 2, 8), datetime.date(1995, 2, 9), datetime.date(1995, 2, 10), datetime.date(1995, 2, 13), datetime.date(1995, 2, 14), datetime.date(1995, 2, 15), datetime.date(1995, 2, 16), datetime.date(1995, 2, 17), datetime.date(1995, 2, 20), datetime.date(1995, 2, 21), dat

In [45]:
str(date_list[0])

'1995-01-03'

In [76]:
# 크롬 옵션즈에 User-Agent, lang 같은 정보를 담아 셀레니움을 이용한 크롤링이 아닌 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')
options.add_argument("lang=ko_KR")

# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

driver.get("https://www.hanabank.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")  # 크롤링 할 웹사이트 주소

In [83]:
new_columns = ('통화','현찰_사실때_환율','현찰_사실때_Spread','현찰_파실때_환율',
               '현찰_파실때_Spread','송금_보낼때','송금_받을때','T/C_사실때',
               '외화_수표_파실때','매매기준율','환가_료율','미화환산율','date')
col_sorted = ('date', '통화','현찰_사실때_환율','현찰_사실때_Spread','현찰_파실때_환율',
               '현찰_파실때_Spread','송금_보낼때','송금_받을때','T/C_사실때',
               '외화_수표_파실때','매매기준율','환가_료율','미화환산율')

In [70]:
for idx, date in enumerate(date_list[:5]):
    # 웹 요소가 나타날 때까지 최대 10초 동안 기다립니다.
    wait = WebDriverWait(driver, 10)

    # 날짜 박스 #tmpInqStrDt
    datebox = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
    datebox.clear()   # 기존에 입력된 값을 지웁니다.
    datebox.send_keys(f"{date}")   # 새로운 값을 입력합니다.

    # 조회버튼   #HANA_CONTENTS_DIV > div.btnBoxCenter > a
    search_box = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
    search_box.click()   # 조회버튼 클릭
    # 생성한 시간 동안 대기합니다.
    time.sleep(random.uniform(3, 7))
    exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchContentDiv > div.printdiv > table')))

    # 페이지의 전체 HTML 소스를 가져옵니다.
    page_html = driver.page_source
    # soup = bs(page_html, "lxml")     # beautifulsoup으로 html을 parsing
    exchange_rate = pd.read_html(page_html)
    exchange_rate = exchange_rate[1]
    exchange_rate['date'] = f"{date}"
    exchange_rate.columns = new_columns
    exchange_rate = exchange_rate[[*col_sorted]]
    
    display(exchange_rate)
    print(f"전체 {len(dates)} 중 {idx+1:04d} 페이지 완료", end="\r")

    

,date,통화,현찰_사실때_환율,현찰_사실때_Spread,현찰_파실때_환율,현찰_파실때_Spread,송금_보낼때,송금_받을때,T/C_파실때,외화 수표 파실때,매매 기준율,환가 료율,미화 환산율
0,1995-01-03,미국 USD,800.53,1.5,776.87,1.5,791.80,785.60,792.64,0.00,788.70,0.00,1.0000
1,1995-01-03,일본 JPY (100),798.89,1.5,775.29,1.5,791.81,782.37,792.59,0.00,787.09,0.00,0.9980
2,1995-01-03,유로 EUR,0.00,0.0,0.00,0.0,975.40,959.92,0.00,957.94,967.66,7.37,1.2269
3,1995-01-03,중국 CNY,0.00,0.0,0.00,0.0,94.12,0.00,0.00,0.00,93.38,1.00,0.1184
4,1995-01-03,홍콩 HKD,103.45,3.0,100.41,3.0,102.74,101.12,0.00,0.00,101.93,0.00,0.1292
5,1995-01-03,태국 THB,32.53,3.0,30.65,3.0,31.84,31.34,0.00,31.26,31.59,11.25,0.0401
6,1995-01-03,싱가포르 SGD,549.06,3.0,532.84,3.0,545.27,536.63,0.00,0.00,540.95,0.00,0.6859
7,1995-01-03,호주 AUD,621.08,3.0,602.74,3.0,616.80,607.02,617.41,0.00,611.91,0.00,0.7758
8,1995-01-03,영국 GBP,1248.38,3.0,1211.50,3.0,1239.77,1220.11,1241.00,0.00,1229.94,0.00,1.5595
9,1995-01-03,캐나다 CAD,571.02,3.0,554.16,3.0,567.09,558.09,567.65,0.00,562.59,0.00,0.7133


,date,통화,현찰_사실때_환율,현찰_사실때_Spread,현찰_파실때_환율,현찰_파실때_Spread,송금_보낼때,송금_받을때,T/C_파실때,외화 수표 파실때,매매 기준율,환가 료율,미화 환산율
0,1995-01-04,미국 USD,800.33,1.5,776.67,1.5,791.60,785.40,792.44,0.00,788.50,0.00,1.0000
1,1995-01-04,일본 JPY (100),794.76,1.5,771.28,1.5,787.71,778.33,788.50,0.00,783.02,0.00,0.9931
2,1995-01-04,유로 EUR,0.00,0.0,0.00,0.0,971.40,956.00,0.00,954.03,963.70,7.37,1.2222
3,1995-01-04,중국 CNY,0.00,0.0,0.00,0.0,94.11,0.00,0.00,0.00,93.37,1.00,0.1184
4,1995-01-04,홍콩 HKD,103.42,3.0,100.38,3.0,102.71,101.09,0.00,0.00,101.90,0.00,0.1292
5,1995-01-04,태국 THB,32.49,3.0,30.61,3.0,31.80,31.30,0.00,31.22,31.55,11.25,0.0400
6,1995-01-04,싱가포르 SGD,550.50,3.0,534.24,3.0,546.70,538.04,0.00,0.00,542.37,0.00,0.6879
7,1995-01-04,호주 AUD,614.92,3.0,596.76,3.0,610.68,601.00,611.29,0.00,605.84,0.00,0.7683
8,1995-01-04,영국 GBP,1251.91,3.0,1214.91,3.0,1243.27,1223.55,1244.51,0.00,1233.41,0.00,1.5642
9,1995-01-04,캐나다 CAD,569.36,3.0,552.54,3.0,565.43,556.47,565.99,0.00,560.95,0.00,0.7114


,date,통화,현찰_사실때_환율,현찰_사실때_Spread,현찰_파실때_환율,현찰_파실때_Spread,송금_보낼때,송금_받을때,T/C_파실때,외화 수표 파실때,매매 기준율,환가 료율,미화 환산율
0,1995-01-05,미국 USD,801.14,1.5,777.46,1.5,792.40,786.20,793.24,0.00,789.30,0.00,1.0000
1,1995-01-05,일본 JPY (100),790.74,1.5,767.38,1.5,783.73,774.39,784.51,0.00,779.06,0.00,0.9870
2,1995-01-05,유로 EUR,0.00,0.0,0.00,0.0,971.92,956.50,0.00,954.53,964.21,7.37,1.2216
3,1995-01-05,중국 CNY,0.00,0.0,0.00,0.0,94.21,0.00,0.00,0.00,93.47,1.00,0.1184
4,1995-01-05,홍콩 HKD,103.48,3.0,100.44,3.0,102.77,101.15,0.00,0.00,101.96,0.00,0.1292
5,1995-01-05,태국 THB,32.51,3.0,30.63,3.0,31.82,31.32,0.00,31.24,31.57,11.25,0.0400
6,1995-01-05,싱가포르 SGD,550.99,3.0,534.71,3.0,547.19,538.51,0.00,0.00,542.85,0.00,0.6878
7,1995-01-05,호주 AUD,616.03,3.0,597.83,3.0,611.78,602.08,612.39,0.00,606.93,0.00,0.7689
8,1995-01-05,영국 GBP,1250.17,3.0,1213.23,3.0,1241.55,1221.85,1242.78,0.00,1231.70,0.00,1.5605
9,1995-01-05,캐나다 CAD,571.44,3.0,554.56,3.0,567.50,558.50,568.06,0.00,563.00,0.00,0.7133


,date,통화,현찰_사실때_환율,현찰_사실때_Spread,현찰_파실때_환율,현찰_파실때_Spread,송금_보낼때,송금_받을때,T/C_파실때,외화 수표 파실때,매매 기준율,환가 료율,미화 환산율
0,1995-01-06,미국 USD,801.44,1.5,777.76,1.5,792.70,786.50,793.54,0.00,789.60,0.00,1.0000
1,1995-01-06,일본 JPY (100),793.90,1.5,770.44,1.5,786.86,777.48,787.64,0.00,782.17,0.00,0.9906
2,1995-01-06,유로 EUR,0.00,0.0,0.00,0.0,976.42,960.94,0.00,958.96,968.68,7.37,1.2268
3,1995-01-06,중국 CNY,0.00,0.0,0.00,0.0,94.26,0.00,0.00,0.00,93.52,1.00,0.1184
4,1995-01-06,홍콩 HKD,103.53,3.0,100.47,3.0,102.81,101.19,0.00,0.00,102.00,0.00,0.1292
5,1995-01-06,태국 THB,32.52,3.0,30.64,3.0,31.83,31.33,0.00,31.25,31.58,11.25,0.0400
6,1995-01-06,싱가포르 SGD,551.84,3.0,535.54,3.0,548.03,539.35,0.00,0.00,543.69,0.00,0.6886
7,1995-01-06,호주 AUD,616.90,3.0,598.68,3.0,612.65,602.93,613.26,0.00,607.79,0.00,0.7697
8,1995-01-06,영국 GBP,1252.25,3.0,1215.25,3.0,1243.62,1223.88,1244.85,0.00,1233.75,0.00,1.5625
9,1995-01-06,캐나다 CAD,572.35,3.0,555.45,3.0,568.41,559.39,568.97,0.00,563.90,0.00,0.7142


,date,통화,현찰_사실때_환율,현찰_사실때_Spread,현찰_파실때_환율,현찰_파실때_Spread,송금_보낼때,송금_받을때,T/C_파실때,외화 수표 파실때,매매 기준율,환가 료율,미화 환산율
0,1995-01-09,미국 USD,802.15,1.5,778.45,1.5,793.40,787.20,794.25,0.00,790.30,0.00,1.0000
1,1995-01-09,일본 JPY (100),792.24,1.5,768.84,1.5,785.22,775.86,786.00,0.00,780.54,0.00,0.9877
2,1995-01-09,유로 EUR,0.00,0.0,0.00,0.0,968.92,953.56,0.00,951.63,961.24,7.25,1.2163
3,1995-01-09,중국 CNY,0.00,0.0,0.00,0.0,94.35,0.00,0.00,0.00,93.61,1.00,0.1184
4,1995-01-09,홍콩 HKD,103.40,3.0,100.36,3.0,102.69,101.07,0.00,0.00,101.88,0.00,0.1289
5,1995-01-09,태국 THB,32.55,3.0,30.67,3.0,31.86,31.36,0.00,31.28,31.61,11.25,0.0400
6,1995-01-09,싱가포르 SGD,551.80,3.0,535.50,3.0,547.99,539.31,0.00,0.00,543.65,0.00,0.6879
7,1995-01-09,호주 AUD,614.08,3.0,595.94,3.0,609.85,600.17,610.45,0.00,605.01,0.00,0.7655
8,1995-01-09,영국 GBP,1244.94,3.0,1208.16,3.0,1236.36,1216.74,1237.58,0.00,1226.55,0.00,1.5520
9,1995-01-09,캐나다 CAD,572.98,3.0,556.06,3.0,569.03,560.01,569.60,0.00,564.52,0.00,0.7143


In [56]:
cols = dict(zip(tuple(exchange_rate.columns), new_columns))

In [61]:
cols

{('통화', '통화', '통화'): '통화',
 ('현찰', '사실 때', '환율'): '현찰_사실때_환율',
 ('현찰', '사실 때', 'Spread'): '현찰_사실때_Spread',
 ('현찰', '파실 때', '환율'): '현찰_파실때_환율',
 ('현찰', '파실 때', 'Spread'): '현찰_파실때_Spread',
 ('송금', '보낼 때', '보낼 때'): '송금_보낼때',
 ('송금', '받을 때', '받을 때'): '송금_받을때',
 ('T/C 사실때', 'T/C 사실때', 'T/C 사실때'): 'T/C_파실때',
 ('외화 수표 파실때', '외화 수표 파실때', '외화 수표 파실때'): '외화 수표 파실때',
 ('매매 기준율', '매매 기준율', '매매 기준율'): '매매 기준율',
 ('환가 료율', '환가 료율', '환가 료율'): '환가 료율',
 ('미화 환산율', '미화 환산율', '미화 환산율'): '미화 환산율',
 ('date', '', ''): 'date'}

In [63]:
exchange_rate = exchange_rate.rename(columns={('통화', '통화', '통화'): '통화',
 ('현찰', '사실 때', '환율'): '현찰_사실때_환율'})
exchange_rate

통화       현찰                              송금           \
                 통화     사실 때            파실 때            보낼 때     받을 때   
                 통화       환율 Spread       환율 Spread     보낼 때     받을 때   
0            미국 USD   802.15    1.5   778.45    1.5   793.40   787.20   
1      일본 JPY (100)   792.24    1.5   768.84    1.5   785.22   775.86   
2            유로 EUR     0.00    0.0     0.00    0.0   968.92   953.56   
3            중국 CNY     0.00    0.0     0.00    0.0    94.35     0.00   
4            홍콩 HKD   103.40    3.0   100.36    3.0   102.69   101.07   
5            태국 THB    32.55    3.0    30.67    3.0    31.86    31.36   
6          싱가포르 SGD   551.80    3.0   535.50    3.0   547.99   539.31   
7            호주 AUD   614.08    3.0   595.94    3.0   609.85   600.17   
8            영국 GBP  1244.94    3.0  1208.16    3.0  1236.36  1216.74   
9           캐나다 CAD   572.98    3.0   556.06    3.0   569.03   560.01   
10        말레이시아 MYR   314.36    3.0   305.08    3.0   312.19   307.25   
11         노르웨이 NOK   117.52    3.0   114.06    3.0   116.71   114.87   
12         뉴질랜드 NZD   510.37    3.0   495.29    3.0   506.85   498.81   
13          덴마크 DKK   130.45    3.0   126.61    3.0   129.55   127.51   
14          바레인 BHD  2128.29    3.0  2065.39    3.0  2113.61  2080.07   
15      사우디아라비아 SAR   213.81    3.0   207.51    3.0   212.34   208.98   
16          스웨덴 SEK   106.26    3.0   103.12    3.0   105.52   103.86   
17          스위스 CHF   612.56    3.0   594.46    3.0   608.33   598.69   
18    아랍에미리트공화국 AED   218.41    3.0   211.97    3.0   216.91   213.47   
19           인도 INR    25.94    3.0    24.44    3.0     0.00     0.00   
20  인도네시아 IDR (100)    37.03    3.0    34.89    3.0    36.24    35.68   
21         쿠웨이트 KWD  2675.63    3.0  2596.55    3.0  2657.17  2615.01   

    T/C 사실때 외화 수표 파실때   매매 기준율  환가 료율  미화 환산율        date  
    T/C 사실때 외화 수표 파실때   매매 기준율  환가 료율  미화 환산율              
    T/C 사실때 외화 수표 파실때   매매 기준율  환가 료율  미화 환산율              
0    794.25      0.00   790.30   0.00  1.0000  1995-01-09  
1    786.00      0.00   780.54   0.00  0.9877  1995-01-09  
2      0.00    951.63   961.24   7.25  1.2163  1995-01-09  
3      0.00      0.00    93.61   1.00  0.1184  1995-01-09  
4      0.00      0.00   101.88   0.00  0.1289  1995-01-09  
5      0.00     31.28    31.61  11.25  0.0400  1995-01-09  
6      0.00      0.00   543.65   0.00  0.6879  1995-01-09  
7    610.45      0.00   605.01   0.00  0.7655  1995-01-09  
8   1237.58      0.00  1226.55   0.00  1.5520  1995-01-09  
9    569.60      0.00   564.52   0.00  0.7143  1995-01-09  
10     0.00      0.00   309.72   0.00  0.3919  1995-01-09  
11     0.00      0.00   115.79   0.00  0.1465  1995-01-09  
12     0.00      0.00   502.83   0.00  0.6363  1995-01-09  
13     0.00      0.00   128.53   0.00  0.1626  1995-01-09  
14     0.00      0.00  2096.84   0.00  2.6532  1995-01-09  
15     0.00      0.00   210.66   0.00  0.2666  1995-01-09  
16     0.00      0.00   104.69   0.00  0.1325  1995-01-09  
17     0.00      0.00   603.51   0.00  0.7636  1995-01-09  
18     0.00      0.00   215.19   0.00  0.2723  1995-01-09  
19     0.00      0.00    25.19   9.25  0.0319  1995-01-09  
20     0.00     35.54    35.96  16.25  0.0455  1995-01-09  
21     0.00      0.00  2636.09   0.00  3.3356  1995-01-09

In [25]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, 0 to 57
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   통화             290 non-null    object 
 1   현찰_사실때_환율      290 non-null    float64
 2   현찰_사실때_Spread  290 non-null    float64
 3   현찰_파실때_환율      290 non-null    float64
 4   현찰_파실때_Spread  290 non-null    float64
 5   송금_보낼때         290 non-null    float64
 6   송금_받을때         290 non-null    float64
 7   T/C_파실때        290 non-null    float64
 8   외화 수표 파실때      290 non-null    float64
 9   매매 기준율         290 non-null    float64
 10  환가 료율          290 non-null    float64
 11  미화 환산율         290 non-null    float64
 12  date           290 non-null    object 
dtypes: float64(11), object(2)
memory usage: 39.8+ KB


In [26]:
final_df

,통화,현찰_사실때_환율,현찰_사실때_Spread,현찰_파실때_환율,현찰_파실때_Spread,송금_보낼때,송금_받을때,T/C_파실때,외화 수표 파실때,매매 기준율,환가 료율,미화 환산율,date
0,미국 USD,1394.99,1.75,1347.01,1.75,1384.40,1357.60,0.0,1354.87,1371.00,7.19233,1.0000,2024-07-31
1,일본 JPY (100),930.30,1.75,898.30,1.75,923.26,905.34,0.0,904.85,914.30,2.19682,0.6669,2024-07-31
2,유로 EUR,1513.99,1.99,1454.91,1.99,1499.29,1469.61,0.0,1467.32,1484.45,5.57100,1.0827,2024-07-31
3,중국 CNY,199.27,5.00,180.31,5.00,191.68,187.90,0.0,0.00,189.79,4.64409,0.1384,2024-07-31
4,홍콩 HKD,178.94,1.97,172.04,1.97,177.24,173.74,0.0,173.46,175.49,6.57366,0.1280,2024-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,리비아 LYD,0.00,0.00,0.00,0.00,289.60,283.88,0.0,0.00,286.74,0.00000,0.2070,2024-07-27
54,루마니아 RON,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,302.61,0.00000,0.2184,2024-07-27
55,미얀마 MMK,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.66,0.00000,0.0005,2024-07-27
56,에티오피아 ETB,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,23.97,0.00000,0.0173,2024-07-27


# DB에 직접 저장

In [35]:
import dbio

In [84]:
for idx, date in enumerate(date_list[:5]):
    # 웹 요소가 나타날 때까지 최대 10초 동안 기다립니다.
    wait = WebDriverWait(driver, 10)

    # 날짜 박스 #tmpInqStrDt
    datebox = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
    datebox.clear()   # 기존에 입력된 값을 지웁니다.
    datebox.send_keys(f"{date}")   # 새로운 값을 입력합니다.

    # 조회버튼   #HANA_CONTENTS_DIV > div.btnBoxCenter > a
    search_box = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
    search_box.click()   # 조회버튼 클릭
    # 생성한 시간 동안 대기합니다.
    time.sleep(random.uniform(3, 7))
    exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchContentDiv > div.printdiv > table')))

    # 페이지의 전체 HTML 소스를 가져옵니다.
    page_html = driver.page_source
    # soup = bs(page_html, "lxml")     # beautifulsoup으로 html을 parsing
    exchange_rate_df = pd.read_html(page_html)
    exchange_rate_df = exchange_rate_df[1]
    exchange_rate_df['date'] = f"{date}"
    exchange_rate_df.columns = new_columns
    exchange_rate_df = exchange_rate_df[[*col_sorted]]
    
    dbio.to_db("exchange_rate_test2", exchange_rate_df)
    print(f"전체 {len(dates)} 중 {idx+1:04d} 페이지 완료", end="\r")

    

In [ ]:
final_df['date'][0]

In [ ]:
to_db("exchange_rate", final_df

In [ ]:
57*365*10

In [ ]:
for mon in range(1,13):
    print(mon)